In [200]:
import pandas as pd
data3=pd.read_csv('anime_planet.csv')
data3.head()

,Unnamed: 0,index,Title,genres,overview,Poster
0,0,0,Attack on Titan The Final Season: The Final Ch...,", Action , Drama , Fantasy , Shounen , Dark F...",Final part ofAttack on Titan The Final Season.,https://cdn.anime-planet.com/anime/primary/att...
1,1,1,Fullmetal Alchemist: Brotherhood,", Action , Adventure , Drama , Fantasy , Myst...",The foundation of alchemy is based on the law ...,https://cdn.anime-planet.com/anime/primary/ful...
2,2,2,Fruits Basket the Final Season,", Drama , Fantasy , Romance , Shoujo , Animal...",The final arc ofFruits Basket.,https://cdn.anime-planet.com/anime/primary/fru...
3,3,3,Demon Slayer: Kimetsu no Yaiba - Entertainment...,", Action , Adventure , Shounen , Demons , His...",Tanjiro and his friends accompany the Hashira ...,https://cdn.anime-planet.com/anime/primary/dem...
4,4,4,Mo Dao Zu Shi 3,", Fantasy , Ancient China , Chinese Animation...",The third season ofMo Dao Zu Shi.,https://cdn.anime-planet.com/anime/primary/mo-...


In [204]:
#data3=data3[['anime_title','anime_overview','anime_genres','anime_poster']]
data3=data3[['Title','overview','genres','Poster']]

In [205]:
data3['tags']=data3['Title']+data3['overview']+data3['genres']
data3['tags']=data3['tags'].str.strip()
data3['tags'][0]

'Attack on Titan The Final Season: The Final ChaptersFinal part ofAttack on Titan The Final Season. , Action , Drama , Fantasy , Shounen , Dark Fantasy , Military , War , Based on a Manga'

In [206]:
data3['title']=data3['Title']
new_data=data3.drop(columns=['overview','genres','Title'])
new_data=new_data[['title','tags','Poster']]
new_data.head()

,title,tags,Poster
0,Attack on Titan The Final Season: The Final Ch...,Attack on Titan The Final Season: The Final Ch...,https://cdn.anime-planet.com/anime/primary/att...
1,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: BrotherhoodThe foundation...,https://cdn.anime-planet.com/anime/primary/ful...
2,Fruits Basket the Final Season,Fruits Basket the Final SeasonThe final arc of...,https://cdn.anime-planet.com/anime/primary/fru...
3,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Demon Slayer: Kimetsu no Yaiba - Entertainment...,https://cdn.anime-planet.com/anime/primary/dem...
4,Mo Dao Zu Shi 3,Mo Dao Zu Shi 3The third season ofMo Dao Zu Sh...,https://cdn.anime-planet.com/anime/primary/mo-...


In [207]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=12248, stop_words='english')
cv


CountVectorizer(max_features=12248, stop_words='english')

In [208]:
vector=cv.fit_transform(new_data['tags'].values.astype('U')).toarray()
vector.shape

(3500, 12248)

In [209]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)
len(similarity[0])

3500

In [210]:
new_data[new_data['title']=='Naruto']


,title,tags,Poster
893,Naruto,NarutoNaruto Uzumaki is a young ninja who bear...,https://cdn.anime-planet.com/anime/primary/nar...


In [211]:
distance = sorted(list(enumerate(similarity[893])), reverse=True, key=lambda vector:vector[1])
for i in distance[0:10]:
    print(new_data.iloc[i[0]].title)

Naruto
Naruto Shippuden Movie 3: The Will of Fire
Naruto Shippuden
Naruto Shippuden Movie 4: The Lost Tower
Naruto Shippuden Movie 2: Bonds
Boruto: Naruto the Movie - The Day Naruto Became Hokage
Naruto Shippuden Movie 5: Blood Prison
Naruto Shippuden OVA: Sage Naruto vs Sasuke
Naruto Shippuden Movie 6: Road to Ninja
Naruto Shippuden: Sunny Side Battle!


In [212]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
new_data['tags'] = new_data['tags'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(new_data['tags'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(3500, 14603)

In [213]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#Construct a reverse map of indices and movie titles
indices = pd.Series(new_data.index, index=new_data['title']).drop_duplicates()

In [214]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return new_data['title'].iloc[movie_indices]

In [215]:
get_recommendations('Naruto')

275                                      Naruto Shippuden
1427    Boruto: Naruto the Movie - The Day Naruto Beca...
1867             Naruto Shippuden Movie 4: The Lost Tower
2061           Naruto Shippuden Movie 3: The Will of Fire
2421                      Naruto Shippuden Movie 2: Bonds
1606                 Naruto Shippuden: Sunny Side Battle!
852               Naruto Shippuden Movie 6: Road to Ninja
1456               Naruto Shippuden Movie 5: Blood Prison
2643          Naruto Shippuden OVA: Sage Naruto vs Sasuke
2310                             Naruto Shippuden Movie 1
Name: title, dtype: object

In [216]:
import pickle
pickle.dump(new_data, open('anime.pkl', 'wb'))
pickle.dump(cosine_sim, open('similarity.pkl', 'wb'))
pickle.load(open('anime.pkl', 'rb'))

,title,tags,Poster
0,Attack on Titan The Final Season: The Final Ch...,Attack on Titan The Final Season: The Final Ch...,https://cdn.anime-planet.com/anime/primary/att...
1,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: BrotherhoodThe foundation...,https://cdn.anime-planet.com/anime/primary/ful...
2,Fruits Basket the Final Season,Fruits Basket the Final SeasonThe final arc of...,https://cdn.anime-planet.com/anime/primary/fru...
3,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Demon Slayer: Kimetsu no Yaiba - Entertainment...,https://cdn.anime-planet.com/anime/primary/dem...
4,Mo Dao Zu Shi 3,Mo Dao Zu Shi 3The third season ofMo Dao Zu Sh...,https://cdn.anime-planet.com/anime/primary/mo-...
...,...,...,...
3495,Doranana x Kirara Magic: Doranana,Doranana x Kirara Magic: DorananaThe offiical ...,https://cdn.anime-planet.com/anime/primary/dor...
3496,Owarimonogatari II Recaps,Owarimonogatari II RecapsNo synopsis yet - che...,https://cdn.anime-planet.com/anime/primary/owa...
3497,Jang Geum Ieui Kkum 2,Jang Geum Ieui Kkum 2No synopsis yet - check b...,https://cdn.anime-planet.com/images/anime/defa...
3498,Sengoku Night Blood: Ongaku Dai Tokushuu,Sengoku Night Blood: Ongaku Dai TokushuuNo syn...,https://cdn.anime-planet.com/anime/primary/sen...
